In [3]:
from pLaplace_1D import test_runner_1D

# all possible types of inputs (not all combinations allowed)
_ = ["numpy", "numba", "jax"]  # "val_grad"
_ = ["numpy_Laplace", "numpy_SFD", "numba_Laplace", "numba_SFD", "jax_dense", "jax_sparse", "jax_Laplace", "jax_SFD"]  # "hess"
_ = ["my_newton", "my_trust_region", "scipy_trust-exact", "scipy_Newton-CG_hess", "scipy_BFGS", "scipy_CG", "scipy_L-BFGS-B", "scipy_TNC"]  # "minimizer"
_ = ["zero", "laplace"]  # "initial_guess"

problem_setting = {"p": 3,   # p in pLaplace
                   "a": -1,  # left bound of computational domain
                   "b": 1,   # right bound of computational domain
                   "f": "default"}  # source term, constant 10

sizes = [10, 100, 1000]


# Conclusions first
- fastest value and grad evaluation have numba (rewrite of matlab code)
- fastest method without the use of user provided hessian is `L-BFGS-B` (scipy)
- method similar to the "best" in the matlab code is `Newton-CG` (scipy) with approximation of hessian using numrical differentiation SFD (rewrite of matlab code)
- fastest method overall is my implementation of `newton` with linesearch using golden section with Laplace as a constant approximation of the hessian 

In [20]:
all_solvers = []
all_solvers.append({"val_grad": "numba",
                    "hess": "numba_SFD",
                    "minimizer": ["scipy_L-BFGS-B", "scipy_Newton-CG_hessp"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "numba_Laplace",
                    "minimizer": ["my_newton"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df = test_runner_1D.create_tables(results, display_table="Ipython")


Iterations:


,10,100,1000
"('numba', 'numba_SFD', 'scipy_L-BFGS-B', 'zero')",14.0,215.0,1775.0
"('numba', 'numba_SFD', 'scipy_Newton-CG_hessp', 'zero')",11.0,15.0,21.0
"('numba', 'numba_Laplace', 'my_newton', 'zero')",12.0,21.0,21.0


Function value:


,10,100,1000
"('numba', 'numba_SFD', 'scipy_L-BFGS-B', 'zero')",-16.772467,-16.864466,-16.865457
"('numba', 'numba_SFD', 'scipy_Newton-CG_hessp', 'zero')",-16.772467,-16.864466,-16.865470
"('numba', 'numba_Laplace', 'my_newton', 'zero')",-16.772467,-16.864464,-16.865467


Compile time:


,10,100,1000
"('numba', 'numba_SFD', 'scipy_L-BFGS-B', 'zero')",0.011482,0.069306,0.356646
"('numba', 'numba_SFD', 'scipy_Newton-CG_hessp', 'zero')",0.011482,0.069306,0.356646
"('numba', 'numba_Laplace', 'my_newton', 'zero')",0.000774,0.000884,0.001306


Solve time:


,10,100,1000
"('numba', 'numba_SFD', 'scipy_L-BFGS-B', 'zero')",0.001992,0.022016,0.236913
"('numba', 'numba_SFD', 'scipy_Newton-CG_hessp', 'zero')",0.021488,0.049873,0.718797
"('numba', 'numba_Laplace', 'my_newton', 'zero')",0.001667,0.007321,0.009955


# Solvers with only grad information
## Comparison of implementations (numpy vs numba vs jax)

In [7]:
all_solvers = []
all_solvers.append({"val_grad": "numpy",
                    "hess": "numpy_Laplace",
                    "minimizer": ["scipy_TNC"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "numba_Laplace",
                    "minimizer": ["scipy_TNC"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_Laplace",
                    "minimizer": ["scipy_TNC"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df = test_runner_1D.create_tables(results, display_table="Ipython")


Iterations:


,10,100,1000
"('numpy', 'numpy_Laplace', 'scipy_TNC', 'zero')",13.0,52.0,522.0
"('numba', 'numba_Laplace', 'scipy_TNC', 'zero')",13.0,57.0,558.0
"('jax', 'jax_Laplace', 'scipy_TNC', 'zero')",11.0,54.0,491.0


Function value:


,10,100,1000
"('numpy', 'numpy_Laplace', 'scipy_TNC', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'numba_Laplace', 'scipy_TNC', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'jax_Laplace', 'scipy_TNC', 'zero')",-16.772467,-16.864466,-16.86547


Compile time:


,10,100,1000
"('numpy', 'numpy_Laplace', 'scipy_TNC', 'zero')",0.002646,0.002714,0.001498
"('numba', 'numba_Laplace', 'scipy_TNC', 'zero')",0.000960,0.002044,0.001471
"('jax', 'jax_Laplace', 'scipy_TNC', 'zero')",0.096916,0.110637,0.100323


Solve time:


,10,100,1000
"('numpy', 'numpy_Laplace', 'scipy_TNC', 'zero')",0.007631,0.077172,1.683145
"('numba', 'numba_Laplace', 'scipy_TNC', 'zero')",0.003735,0.041397,0.758558
"('jax', 'jax_Laplace', 'scipy_TNC', 'zero')",0.006789,0.087619,1.599039


## For numba a comparison of multiple minimizators

In [8]:
all_solvers = []
all_solvers.append({"val_grad": "numba",
                    "hess": "numba_Laplace",
                    "minimizer": ["scipy_TNC", "scipy_BFGS", "scipy_CG", "scipy_L-BFGS-B"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df = test_runner_1D.create_tables(results, display_table="Ipython")


Iterations:


,10,100,1000
"('numba', 'numba_Laplace', 'scipy_TNC', 'zero')",13.0,57.0,558.0
"('numba', 'numba_Laplace', 'scipy_BFGS', 'zero')",21.0,121.0,1000.0
"('numba', 'numba_Laplace', 'scipy_CG', 'zero')",40.0,958.0,26426.0
"('numba', 'numba_Laplace', 'scipy_L-BFGS-B', 'zero')",14.0,215.0,1775.0


Function value:


,10,100,1000
"('numba', 'numba_Laplace', 'scipy_TNC', 'zero')",-16.772467,-16.864466,-16.865470
"('numba', 'numba_Laplace', 'scipy_BFGS', 'zero')",-16.772467,-16.864466,-16.865470
"('numba', 'numba_Laplace', 'scipy_CG', 'zero')",-16.772467,-16.864466,-16.865470
"('numba', 'numba_Laplace', 'scipy_L-BFGS-B', 'zero')",-16.772467,-16.864466,-16.865457


Compile time:


,10,100,1000
"('numba', 'numba_Laplace', 'scipy_TNC', 'zero')",0.002485,0.001051,0.001141
"('numba', 'numba_Laplace', 'scipy_BFGS', 'zero')",0.002485,0.001051,0.001141
"('numba', 'numba_Laplace', 'scipy_CG', 'zero')",0.002485,0.001051,0.001141
"('numba', 'numba_Laplace', 'scipy_L-BFGS-B', 'zero')",0.002485,0.001051,0.001141


Solve time:


,10,100,1000
"('numba', 'numba_Laplace', 'scipy_TNC', 'zero')",0.004142,0.031544,0.750259
"('numba', 'numba_Laplace', 'scipy_BFGS', 'zero')",0.005037,0.169024,129.806057
"('numba', 'numba_Laplace', 'scipy_CG', 'zero')",0.007336,0.094882,4.643936
"('numba', 'numba_Laplace', 'scipy_L-BFGS-B', 'zero')",0.000847,0.008708,0.271462


# Solvers using hessians 
**in scipy**
- `trust-exact` needs dense matrix, no reasonable warkaround
- `newton-cg` needs hessian-vector product

**own implementation of trust region method**

## Comparison of multiple implementation of Hessian approximation
- exact from jax autodiff - both dense and sparse version
- approx using SDF (copy of matlab code)
- constant approximation using Laplace (p=2)

In [13]:
all_solvers = []
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_dense",
                    "minimizer": ["scipy_trust-exact", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_sparse",
                    "minimizer": ["my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_SFD",
                    "minimizer": ["my_trust_region", "scipy_Newton-CG_hessp"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_Laplace",
                    "minimizer": ["my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df = test_runner_1D.create_tables(results, display_table="Ipython")


Iterations:


,10,100,1000
"('jax', 'jax_dense', 'scipy_trust-exact', 'zero')",10.0,55.0,502.0
"('jax', 'jax_dense', 'my_trust_region', 'zero')",8.0,44.0,487.0
"('jax', 'jax_sparse', 'my_trust_region', 'zero')",8.0,44.0,481.0
"('jax', 'jax_SFD', 'my_trust_region', 'zero')",9.0,14.0,28.0
"('jax', 'jax_SFD', 'scipy_Newton-CG_hessp', 'zero')",11.0,17.0,18.0
"('jax', 'jax_Laplace', 'my_trust_region', 'zero')",11.0,25.0,26.0


Function value:


,10,100,1000
"('jax', 'jax_dense', 'scipy_trust-exact', 'zero')",-16.772467,-16.864466,-16.865470
"('jax', 'jax_dense', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.865470
"('jax', 'jax_sparse', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.865470
"('jax', 'jax_SFD', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.865470
"('jax', 'jax_SFD', 'scipy_Newton-CG_hessp', 'zero')",-16.772467,-16.864466,-16.865470
"('jax', 'jax_Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864463,-16.865464


Compile time:


,10,100,1000
"('jax', 'jax_dense', 'scipy_trust-exact', 'zero')",0.471199,0.331090,0.381237
"('jax', 'jax_dense', 'my_trust_region', 'zero')",0.471199,0.331090,0.381237
"('jax', 'jax_sparse', 'my_trust_region', 'zero')",0.359845,0.685522,0.731440
"('jax', 'jax_SFD', 'my_trust_region', 'zero')",0.124278,0.168208,0.541187
"('jax', 'jax_SFD', 'scipy_Newton-CG_hessp', 'zero')",0.124278,0.168208,0.541187
"('jax', 'jax_Laplace', 'my_trust_region', 'zero')",0.103734,0.110064,0.113348


Solve time:


,10,100,1000
"('jax', 'jax_dense', 'scipy_trust-exact', 'zero')",0.014181,0.499928,86.541227
"('jax', 'jax_dense', 'my_trust_region', 'zero')",0.148906,0.904755,51.536381
"('jax', 'jax_sparse', 'my_trust_region', 'zero')",0.077152,0.565665,9.753319
"('jax', 'jax_SFD', 'my_trust_region', 'zero')",0.195600,0.333850,0.859969
"('jax', 'jax_SFD', 'scipy_Newton-CG_hessp', 'zero')",0.087167,0.171173,0.664147
"('jax', 'jax_Laplace', 'my_trust_region', 'zero')",0.078890,0.193050,0.221297


# Comparison of implementations (numpy vs numba vs jax)
 - for trust region
 - also for newton method as both SFD and Laplace converge

In [14]:
all_solvers = []
all_solvers.append({"val_grad": "numpy",
                    "hess": "numpy_Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "numba_Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numpy",
                    "hess": "numpy_SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "numba_SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df = test_runner_1D.create_tables(results, display_table="Ipython")


Iterations:


,10,100,1000
"('numpy', 'numpy_Laplace', 'my_newton', 'zero')",12.0,21.0,21.0
"('numpy', 'numpy_Laplace', 'my_trust_region', 'zero')",11.0,25.0,26.0
"('numba', 'numba_Laplace', 'my_newton', 'zero')",12.0,21.0,21.0
"('numba', 'numba_Laplace', 'my_trust_region', 'zero')",11.0,25.0,26.0
"('jax', 'jax_Laplace', 'my_newton', 'zero')",12.0,21.0,21.0
"('jax', 'jax_Laplace', 'my_trust_region', 'zero')",11.0,25.0,26.0
"('numpy', 'numpy_SFD', 'my_newton', 'zero')",8.0,10.0,69.0
"('numpy', 'numpy_SFD', 'my_trust_region', 'zero')",9.0,16.0,35.0
"('numba', 'numba_SFD', 'my_newton', 'zero')",8.0,10.0,69.0
"('numba', 'numba_SFD', 'my_trust_region', 'zero')",9.0,16.0,36.0


Function value:


,10,100,1000
"('numpy', 'numpy_Laplace', 'my_newton', 'zero')",-16.772467,-16.864464,-16.865467
"('numpy', 'numpy_Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864463,-16.865464
"('numba', 'numba_Laplace', 'my_newton', 'zero')",-16.772467,-16.864464,-16.865467
"('numba', 'numba_Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864463,-16.865464
"('jax', 'jax_Laplace', 'my_newton', 'zero')",-16.772467,-16.864464,-16.865467
"('jax', 'jax_Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864463,-16.865464
"('numpy', 'numpy_SFD', 'my_newton', 'zero')",-16.772467,-16.864466,-16.865470
"('numpy', 'numpy_SFD', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.865470
"('numba', 'numba_SFD', 'my_newton', 'zero')",-16.772467,-16.864466,-16.865470
"('numba', 'numba_SFD', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.865470


Compile time:


,10,100,1000
"('numpy', 'numpy_Laplace', 'my_newton', 'zero')",0.002600,0.000940,0.002918
"('numpy', 'numpy_Laplace', 'my_trust_region', 'zero')",0.002600,0.000940,0.002918
"('numba', 'numba_Laplace', 'my_newton', 'zero')",0.002413,0.000719,0.001685
"('numba', 'numba_Laplace', 'my_trust_region', 'zero')",0.002413,0.000719,0.001685
"('jax', 'jax_Laplace', 'my_newton', 'zero')",0.144465,0.135446,0.132644
"('jax', 'jax_Laplace', 'my_trust_region', 'zero')",0.144465,0.135446,0.132644
"('numpy', 'numpy_SFD', 'my_newton', 'zero')",0.004333,0.041815,0.391426
"('numpy', 'numpy_SFD', 'my_trust_region', 'zero')",0.004333,0.041815,0.391426
"('numba', 'numba_SFD', 'my_newton', 'zero')",0.003879,0.045310,0.427412
"('numba', 'numba_SFD', 'my_trust_region', 'zero')",0.003879,0.045310,0.427412


Solve time:


,10,100,1000
"('numpy', 'numpy_Laplace', 'my_newton', 'zero')",0.010255,0.031500,0.102559
"('numpy', 'numpy_Laplace', 'my_trust_region', 'zero')",0.050106,0.187084,0.254996
"('numba', 'numba_Laplace', 'my_newton', 'zero')",0.003558,0.007373,0.010277
"('numba', 'numba_Laplace', 'my_trust_region', 'zero')",0.031287,0.058904,0.081071
"('jax', 'jax_Laplace', 'my_newton', 'zero')",0.048432,0.084359,0.100031
"('jax', 'jax_Laplace', 'my_trust_region', 'zero')",0.079124,0.228781,0.229809
"('numpy', 'numpy_SFD', 'my_newton', 'zero')",0.009659,0.016758,0.230727
"('numpy', 'numpy_SFD', 'my_trust_region', 'zero')",0.038237,0.136222,0.351145
"('numba', 'numba_SFD', 'my_newton', 'zero')",0.008229,0.011759,0.155541
"('numba', 'numba_SFD', 'my_trust_region', 'zero')",0.028881,0.056325,0.214610


# Initial guess as solution of Laplace (p=2)
Exact hessian (in jax autodiff) very much strougle when starting from 0. It is in orders of magnitude faster when starting from initial guess as solution of Laplace (p=2) (which is cheap in comparison).

In [17]:
all_solvers = []
all_solvers.append({"val_grad": "numba",
                    "hess": "numba_Laplace",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_Laplace",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "numba_SFD",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_SFD",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "jax_sparse",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df = test_runner_1D.create_tables(results, display_table="Ipython")


Iterations:


,10,100,1000
"('numba', 'numba_Laplace', 'my_newton', 'laplace')",7.0,19.0,19.0
"('jax', 'jax_Laplace', 'my_newton', 'laplace')",7.0,19.0,19.0
"('numba', 'numba_SFD', 'my_newton', 'laplace')",4.0,5.0,21.0
"('jax', 'jax_SFD', 'my_newton', 'laplace')",4.0,5.0,19.0
"('jax', 'jax_sparse', 'my_newton', 'laplace')",4.0,5.0,5.0


Function value:


,10,100,1000
"('numba', 'numba_Laplace', 'my_newton', 'laplace')",-16.772467,-16.864464,-16.865467
"('jax', 'jax_Laplace', 'my_newton', 'laplace')",-16.772467,-16.864464,-16.865467
"('numba', 'numba_SFD', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.865470
"('jax', 'jax_SFD', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.865470
"('jax', 'jax_sparse', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.865470


Compile time:


,10,100,1000
"('numba', 'numba_Laplace', 'my_newton', 'laplace')",0.005111,0.001293,0.001516
"('jax', 'jax_Laplace', 'my_newton', 'laplace')",0.152524,0.197721,0.159283
"('numba', 'numba_SFD', 'my_newton', 'laplace')",0.006660,0.040696,0.529483
"('jax', 'jax_SFD', 'my_newton', 'laplace')",0.133756,0.183179,0.614557
"('jax', 'jax_sparse', 'my_newton', 'laplace')",0.347101,0.342761,0.408204


Solve time:


,10,100,1000
"('numba', 'numba_Laplace', 'my_newton', 'laplace')",0.002228,0.003958,0.013614
"('jax', 'jax_Laplace', 'my_newton', 'laplace')",0.025008,0.102066,0.082461
"('numba', 'numba_SFD', 'my_newton', 'laplace')",0.005585,0.007834,0.052908
"('jax', 'jax_SFD', 'my_newton', 'laplace')",0.067386,0.086536,0.482665
"('jax', 'jax_sparse', 'my_newton', 'laplace')",0.009525,0.012619,0.022680
